#Cargo los archivos

In [1]:
import pandas as pd

parquet_file = '/content/drive/MyDrive/Colab Notebooks/TP2/train.parq'

train_data = pd.read_parquet(parquet_file)

train_data

,attack_time,watcher_country,watcher_as_num,watcher_as_name,attacker_country,attacker_as_num,attacker_as_name,attack_type,watcher_uuid_enum,attacker_ip_enum,label
0,2023-07-31 07:17:51+00:00,DE,34011.0,Host Europe GmbH,TR,47721.0,Murat Aktas,http:exploit,0,6466,0
1,2023-07-31 07:17:51+00:00,DE,34011.0,Host Europe GmbH,TR,47721.0,Murat Aktas,http:spam,0,6466,0
2,2023-07-31 07:17:49+00:00,DE,20886.0,bn:t Blatzheim Networks Telecom GmbH,DE,51167.0,Contabo GmbH,http:bruteforce,2,4637,0
3,2023-07-31 07:17:49+00:00,DE,20886.0,bn:t Blatzheim Networks Telecom GmbH,DE,51167.0,Contabo GmbH,http:spam,2,4637,0
4,2023-07-31 07:17:49+00:00,DE,20886.0,bn:t Blatzheim Networks Telecom GmbH,DE,51167.0,Contabo GmbH,http:exploit,2,4637,0
...,...,...,...,...,...,...,...,...,...,...,...
61629680,2023-08-24 21:14:52+00:00,DE,12897.0,ENTEGA Medianet GmbH,DE,3209.0,Vodafone GmbH,http:scan,24879,190784,0
61629681,2023-08-24 21:14:46+00:00,US,22612.0,NAMECHEAP-NET,US,23033.0,WOW,http:scan,40383,193446,0
61629682,2023-08-24 21:14:46+00:00,US,22612.0,NAMECHEAP-NET,US,23033.0,WOW,http:spam,40383,193446,0
61629683,2023-08-24 21:14:46+00:00,US,22612.0,NAMECHEAP-NET,US,23033.0,WOW,http:spam,40383,193446,0


#Data_set

Eliminacion de duplicados

In [2]:
batch_size = 1000000 #Un millón
num_batches = len(train_data) // batch_size #Establece la cantidad de batches que voy a necesitar

duplicated_data = []

for i in range(num_batches):
    batch = train_data[i * batch_size : (i + 1) * batch_size] #Establece de donde hasta donde voy a tomar los valores
    duplicated_batch = batch.drop_duplicates() #Dropea los duplicados dentro de esos valores
    duplicated_data.append(duplicated_batch)

train_data = pd.concat(duplicated_data)
del duplicated_batch
del batch

Reasignacion de valores nulos

Noto que los valores que ocupan el primer puesto en cada columna lo hacen con diferencia por lo que decido que antes que eliminar los valores Nan podria atribuirles estos a las columnas con mayor peso

In [3]:
listaVariables = train_data.count() #Concluyo que watcher_country, watcher_as_name, attacker_country, attacker_as_num y attacker_as_name tienen valores Nan
variables = [listaVariables.index[1], listaVariables.index[3], listaVariables.index[4], listaVariables.index[5], listaVariables.index[6]]

for i in range(5):
  values = train_data[variables[i]].value_counts()
  first_value = values.index[0]
  train_data[variables[i]] = train_data[variables[i]].fillna(first_value)

del listaVariables

In [4]:
train_data.count() #Se reemplazaron correctamente.

attack_time          40041188
watcher_country      40041188
watcher_as_num       40041188
watcher_as_name      40041188
attacker_country     40041188
attacker_as_num      40041188
attacker_as_name     40041188
attack_type          40041188
watcher_uuid_enum    40041188
attacker_ip_enum     40041188
label                40041188
dtype: int64

Agrupacion de categorias con bajo valor

Por otro lado, hay categorias que aparecen una vez contra otras que aparecen mas de un millon de veces por lo que agrupamos estas pequeñas categorias en forma de "otros"

In [5]:
listaVariables = train_data.columns.tolist()
variables1 = [listaVariables[1],  listaVariables[4]] #watcher_country, attacker_country
variables2 = [listaVariables[3], listaVariables[6]] # #watcher_as_name y attacker_as_name

for i in range(2):
  #Se tomó 1.000 como referencia en todos ya que, despues de comprobar los valores, por debajo de eso se consideró insignificante respecto a los demas.
  value_counts = train_data[variables1[i]].value_counts()

  # Encuentra los valores que aparecen menos de 1000 veces
  valores_menos_recuentes = value_counts[value_counts < 1000].index

  # Sustituye estos valores por 'otros'
  train_data[variables1[i]] = train_data[variables1[i]].cat.add_categories('otros')
  train_data.loc[train_data[variables1[i]].isin(valores_menos_recuentes), variables1[i]] = 'otros'

  #Elimina las categorias que no tienen ninguna valor
  train_data[variables1[i]].cat.remove_unused_categories(inplace=True)

del variables1
del value_counts
del valores_menos_recuentes

for i in range(2):
  #Se tomó 25.000 como referencia en todos ya que, despues de comprobar los valores, por debajo de eso se consideró insignificante respecto a los demas.
  value_counts = train_data[variables2[i]].value_counts()

  # Encuentra los valores que aparecen menos de 25.000 veces
  valores_menos_recuentes = value_counts[value_counts < 25000].index

  # Sustituye estos valores por 'otros'
  train_data[variables2[i]] = train_data[variables2[i]].cat.add_categories('otros')
  train_data.loc[train_data[variables2[i]].isin(valores_menos_recuentes), variables2[i]] = 'otros'

  #Elimina las categorias que no tienen ninguna valor
  train_data[variables2[i]].cat.remove_unused_categories(inplace=True)

del variables2
del value_counts
del valores_menos_recuentes

<ipython-input-5-184de54c82db>:17: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  train_data[variables1[i]].cat.remove_unused_categories(inplace=True)
<ipython-input-5-184de54c82db>:17: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  train_data[variables1[i]].cat.remove_unused_categories(inplace=True)
<ipython-input-5-184de54c82db>:35: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  train_data[variables2[i]].cat.remove_unused_categories(inplace=True)
<ipython-input-5-184de54c82db>:35: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  train_data[variables2[i]].cat.remove_unused_categories(inplace=True)


Distribucion de las categorias con menor valor hacia las de mayor valor

Consideré util que en el caso en el que haya muchos valores agrupados dentro de "otros" como sucede con las columnas as_name, me conviene mas distribuir equitativamente los valores asociados a esas filas con lsa filas del TOP de cada columna

In [6]:
list_counts = train_data['watcher_as_name'].value_counts()

import math
#El valor list_counts[0] hace referencia a la cantidad de elementos que tiene la categoria 'otros'
cantidad_distribuciones = math.ceil(list_counts[0] / 25)

for iteracion in range(25):
  values = train_data['watcher_as_name'].value_counts()
  categoria_a_cambiar = values.index[iteracion]
  elementos_a_cambiar = train_data['watcher_as_name'][train_data['watcher_as_name'] == 'otros'].iloc[0:cantidad_distribuciones]
  train_data['watcher_as_name'].loc[elementos_a_cambiar.index] = categoria_a_cambiar

train_data['watcher_as_name'].cat.remove_unused_categories(inplace=True)
del list_counts
del categoria_a_cambiar
del elementos_a_cambiar


list_counts = train_data['attacker_as_name'].value_counts()


import math
#El valor list_counts[0] hace referencia a la cantidad de elementos que tiene la categoria 'otros'
cantidad_distribuciones = math.ceil(list_counts[0] / 25)

for iteracion in range(25):
  values = train_data['attacker_as_name'].value_counts()
  categoria_a_cambiar = values.index[iteracion]
  elementos_a_cambiar = train_data['attacker_as_name'][train_data['attacker_as_name'] == 'otros'].iloc[0:cantidad_distribuciones]
  train_data['attacker_as_name'].loc[elementos_a_cambiar.index] = categoria_a_cambiar

train_data['attacker_as_name'].cat.remove_unused_categories(inplace=True)
del list_counts
del categoria_a_cambiar
del elementos_a_cambiar

del listaVariables

<ipython-input-6-5fbde7c4b8fe>:13: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  train_data['watcher_as_name'].cat.remove_unused_categories(inplace=True)
<ipython-input-6-5fbde7c4b8fe>:32: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  train_data['attacker_as_name'].cat.remove_unused_categories(inplace=True)


Redistribuyo los valores de attack_type que resultan desconocidos entre los primeros valores que toman mas relevancia

In [7]:
list_counts = train_data['attack_type'].value_counts()
list_counts

http:spam               12891106
http:scan                9368903
http:exploit             7135743
ssh:bruteforce           4475835
http:bruteforce          4193622
windows:bruteforce        563787
pop3/imap:bruteforce      436314
tcp:scan                  332971
unknown:unknown           317859
sip:bruteforce            222870
http:crawl                 86263
smb:bruteforce             12203
telnet:bruteforce           2310
ftp:bruteforce               949
database:bruteforce          453
Name: attack_type, dtype: int64

In [8]:
import math

#El valor list_counts[8] hace referencia a la cantidad de elementos que tiene la categoria 'unknonwn,unknown'
cantidad_distribuciones = math.ceil(list_counts[8] / 3)

for iteracion in range(3):
  values = train_data['attack_type'].value_counts()
  categoria_a_cambiar = values.index[iteracion]
  elementos_a_cambiar = train_data['attack_type'][train_data['attack_type'] == 'unknown:unknown'].iloc[0:cantidad_distribuciones]
  train_data['attack_type'].loc[elementos_a_cambiar.index] = categoria_a_cambiar

train_data['attack_type'].cat.remove_unused_categories(inplace=True)
del list_counts
del categoria_a_cambiar
del elementos_a_cambiar

<ipython-input-8-1705459b9486>:12: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  train_data['attack_type'].cat.remove_unused_categories(inplace=True)


Ordeno el df para realizar un split al momento de querer separar mi TRAIN de VALIDATION

In [9]:
train_data = train_data.sort_values(by=['attacker_ip_enum','attack_time'])
train_data

,attack_time,watcher_country,watcher_as_num,watcher_as_name,attacker_country,attacker_as_num,attacker_as_name,attack_type,watcher_uuid_enum,attacker_ip_enum,label
2746887,2023-08-11 14:25:11+00:00,US,29709.0,OVH SAS,US,14618.0,AMAZON-AES,http:spam,18401,0,0
2746888,2023-08-11 14:25:11+00:00,US,29709.0,OVH SAS,US,14618.0,AMAZON-AES,http:scan,18401,0,0
3704781,2023-08-11 14:39:39+00:00,DE,24961.0,myLoc managed IT AG,US,14618.0,AMAZON-AES,http:spam,48687,0,0
3704782,2023-08-11 14:39:39+00:00,DE,24961.0,myLoc managed IT AG,US,14618.0,AMAZON-AES,http:scan,48687,0,0
3535768,2023-08-11 14:46:17+00:00,US,137870.0,Hetzner Online GmbH,US,14618.0,AMAZON-AES,http:spam,2838,0,0
...,...,...,...,...,...,...,...,...,...,...,...
60187702,2023-08-12 21:00:05+00:00,SG,20473.0,AS-CHOOPA,IL,6810.0,otros,http:exploit,211,199965,0
60187705,2023-08-12 21:00:05+00:00,SG,20473.0,AS-CHOOPA,IL,6810.0,otros,http:scan,211,199965,0
60045114,2023-08-12 21:07:26+00:00,SG,20473.0,AS-CHOOPA,IL,6810.0,otros,http:scan,211,199965,0
60045115,2023-08-12 21:07:26+00:00,SG,20473.0,AS-CHOOPA,IL,6810.0,otros,http:spam,211,199965,0


Convirto la hora en un formato que considero sea mas util ya que actualmente todas las variables de attack_time ocurren en el mismo año y solo suceden dentro de un periodo de dos meses. Tambien considero que los segundos son inncesearios.

Ademas, siendo que attacker_as_name y attacker_as_num repersentan lo mismo, decido quedarmo con las partes de as_name y dropear las as_num para no tener informacion redundante. El motivo por el que decidí quedarme con as_name fué porque mostró unos mejores resultados al entrenar el modelo.

In [10]:
train_data = train_data.drop(columns=['attacker_as_num','watcher_as_num'])
train_data['attack_time'] = train_data['attack_time'].dt.hour + train_data['attack_time'].dt.minute / 60.0

Cambio un poco el espacio que ocupan algunas columnas para que mi data_set sea menos pesado para la RAM.

In [11]:
train_data['attack_time'] = pd.to_numeric(train_data['attack_time'], downcast='float')
train_data['watcher_uuid_enum'] = pd.to_numeric(train_data['watcher_uuid_enum'], downcast='integer')
train_data['attacker_ip_enum'] = pd.to_numeric(train_data['attacker_ip_enum'], downcast='integer')

Almaceno el dataset analizado.

In [12]:
train_data.to_parquet('/content/drive/MyDrive/Colab Notebooks/TP2/Perceptron(Baseline)/train_set_analizado.parq')

del train_data

#Kaggle_Set

In [13]:
parquet_file = '/content/drive/MyDrive/Colab Notebooks/TP2/test.parq'

test_data = pd.read_parquet(parquet_file)

test_data

,attack_time,watcher_country,watcher_as_num,watcher_as_name,attacker_country,attacker_as_num,attacker_as_name,attack_type,watcher_uuid_enum,attacker_ip_enum
0,2023-07-31 07:17:58+00:00,US,54641.0,IMH-IAD,IN,14061.0,DIGITALOCEAN-ASN,http:spam,1,7696
1,2023-07-31 07:17:58+00:00,US,54641.0,IMH-IAD,IN,14061.0,DIGITALOCEAN-ASN,http:exploit,1,7696
2,2023-07-31 07:17:58+00:00,US,54641.0,IMH-IAD,IN,14061.0,DIGITALOCEAN-ASN,http:bruteforce,1,7696
3,2023-07-31 07:17:58+00:00,US,54641.0,IMH-IAD,IN,14061.0,DIGITALOCEAN-ASN,http:bruteforce,1,7696
4,2023-07-31 07:17:51+00:00,US,396982.0,GOOGLE-CLOUD-PLATFORM,US,53667.0,PONYNET,http:bruteforce,3,7543
...,...,...,...,...,...,...,...,...,...,...
18682292,2023-08-24 21:14:42+00:00,DE,12897.0,ENTEGA Medianet GmbH,DE,3320.0,Deutsche Telekom AG,http:scan,24879,198081
18682293,2023-08-24 21:14:52+00:00,DE,12897.0,ENTEGA Medianet GmbH,DE,3320.0,Deutsche Telekom AG,http:scan,24879,198081
18682294,2023-08-24 21:14:42+00:00,DE,12897.0,ENTEGA Medianet GmbH,DE,3320.0,Deutsche Telekom AG,http:scan,24879,198081
18682295,2023-08-24 21:14:42+00:00,DE,12897.0,ENTEGA Medianet GmbH,DE,3320.0,Deutsche Telekom AG,http:scan,24879,198081


Aplico las nociones basicas de lo que hice arriba pero en el set de Kaggle, sin modificarlo o quitarle informacion

In [14]:
listaVariables = test_data.count() #Concluyo que watcher_country, watcher_as_name, attacker_country, attacker_as_num y attacker_as_name tienen valores Nan
variables = [listaVariables.index[1], listaVariables.index[3], listaVariables.index[4], listaVariables.index[5], listaVariables.index[6]]

for i in range(5):
  values = test_data[variables[i]].value_counts()
  first_value = values.index[0]
  test_data[variables[i]] = test_data[variables[i]].fillna(first_value)

del listaVariables

test_data = test_data.drop(columns=['attacker_as_num','watcher_as_num'])
test_data['attack_time'] = test_data['attack_time'].dt.hour + test_data['attack_time'].dt.minute / 60.0

test_data['attack_time'] = pd.to_numeric(test_data['attack_time'], downcast='float')
test_data['watcher_uuid_enum'] = pd.to_numeric(test_data['watcher_uuid_enum'], downcast='integer')
test_data['attacker_ip_enum'] = pd.to_numeric(test_data['attacker_ip_enum'], downcast='integer')

In [15]:
test_data.to_parquet('/content/drive/MyDrive/Colab Notebooks/TP2/Perceptron(Baseline)/test_set_analizado.parq')